In [1]:
import pandas as pd
import os
import numpy as np
!pip install --upgrade tensorflow
import tensorflow as tf

# Force TensorFlow to use CPU only
tf.config.set_visible_devices([], 'GPU')

!pip install --upgrade tensorflow==2.15.0.post1
!pip install --upgrade packaging==23.2
!pip install --upgrade shapely==2.0.2
!pip install --upgrade jupyter-lsp


from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.0 MB/s eta 0:00:00
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflowjs 4.16.0 requires packaging~=23.1, but you have packaging 21.3 which is incompatible.


2024-02-04 18:23:51.770064: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-04 18:23:51.770217: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-04 18:23:51.773761: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 23.2 which is incompatible.
jupyterlab 4.0.11 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.0.2 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
osmnx 1.8.1 requires shapely>=2.0, but you have shapely 1.8.5.post1 which is inc

In [2]:
# Set the path to your dataset
data_path = '/kaggle/input/5-flower-types-classification-dataset/flower_images'
categories = os.listdir(data_path)

In [3]:
# Create empty lists to store file paths and corresponding labels
X, y = [], []

In [4]:
# Load file paths and labels
for category in categories:
    path = os.path.join(data_path, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        X.append(img_path)
        y.append(category)

In [5]:
# Split the data into training, testing, and validation sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=101)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=101)

In [6]:
# Create DataFrames for training, testing, and validation sets
train_df = pd.DataFrame({'FileName': X_train, 'Category': y_train})
test_df = pd.DataFrame({'FileName': X_test, 'Category': y_test})
val_df = pd.DataFrame({'FileName': X_val, 'Category': y_val})

In [7]:
# Create image data generators for training, testing, and validation sets with augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
training_set = train_datagen.flow_from_dataframe(train_df, x_col='FileName', y_col='Category',
                                                 batch_size=32, class_mode='categorical', target_size=(64, 64))
test_set = test_datagen.flow_from_dataframe(test_df, x_col='FileName', y_col='Category',
                                            batch_size=32, class_mode='categorical', target_size=(64, 64))
validation_set = test_datagen.flow_from_dataframe(val_df, x_col='FileName', y_col='Category',
                                                   batch_size=32, class_mode='categorical', target_size=(64, 64))

Found 3500 validated image filenames belonging to 5 classes.
Found 750 validated image filenames belonging to 5 classes.
Found 750 validated image filenames belonging to 5 classes.


In [9]:
# Build the CNN model
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=len(categories), activation='softmax'))

In [10]:
# Compile the model
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
# Train the model
cnn.fit(x=training_set, validation_data=validation_set, epochs=100)

Epoch 1/100
110/110 [==============================] - 38s 335ms/step - loss: 1.3025 - accuracy: 0.4594 - val_loss: 1.1061 - val_accuracy: 0.5760
Epoch 2/100
110/110 [==============================] - 27s 242ms/step - loss: 1.1398 - accuracy: 0.5403 - val_loss: 1.0231 - val_accuracy: 0.5853
Epoch 3/100
110/110 [==============================] - 27s 241ms/step - loss: 1.0605 - accuracy: 0.5831 - val_loss: 0.9912 - val_accuracy: 0.6080
Epoch 4/100
110/110 [==============================] - 24s 220ms/step - loss: 1.0158 - accuracy: 0.5926 - val_loss: 0.9855 - val_accuracy: 0.6093
Epoch 5/100
110/110 [==============================] - 24s 221ms/step - loss: 0.9986 - accuracy: 0.6046 - val_loss: 0.9275 - val_accuracy: 0.6400
Epoch 6/100
110/110 [==============================] - 27s 242ms/step - loss: 0.9624 - accuracy: 0.6260 - val_loss: 0.8784 - val_accuracy: 0.6467
Epoch 7/100
110/110 [==============================] - 26s 239ms/step - loss: 0.8995 - accuracy: 0.6429 - val_loss: 0.8381 -

In [12]:
# Evaluate the model on the test set
cnn.evaluate(test_set)

24/24 [==============================] - 7s 300ms/step - loss: 0.6738 - accuracy: 0.8133


[0.6737871766090393, 0.8133333325386047]